In [1]:
from CAI import CAI
import pandas as pd

D:\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
N315_data = pd.read_csv("../dataset/N315.csv")
# 替换值
N315_data['essential'] = N315_data['essential'].replace({'yes': 1, 'no': 0})

N315_data = N315_data[(N315_data['essential'] == 0) | (N315_data['essential'] == 1)]

print(N315_data)

     locus tag essential                                                DNA  \
0       SA0001         1  ATGTCGGAAAAAGAAATTTGGGAAAAAGTGCTTGAAATTGCTCAAG...   
1       SA0002         0  ATGATGGAATTCACTATTAAAAGAGATTATTTTATTACACAATTAA...   
2       SA0003         0  GTGATTATTTTGGTTCAAGAAGTTGTAGTAGAAGGAGACATTAATT...   
3       SA0004         1  ATGAAGTTAAATACACTCCAATTAGAAAATTATCGTAACTATGATG...   
4       SA0005         1  ATGGTGACTGCATTGTCAGATGTAAACAACACGGATAATTATGGTG...   
...        ...       ...                                                ...   
2661    SA2498         0  ATGAAAAAACCTTTTTCAAAATTATTTGGTTTGAAAAACAAAGATG...   
2662    SA2499         1  ATGACTGTAGAATGGTTAGCAGAACAATTAAAAGAACATAATATTG...   
2663    SA2500         1  GTGGTTCAAGAATATGATGTAATCGTTATAGGTGCGGGACATGCAG...   
2665    SA2502         1  TTGGAAAAAGCTTACCGAATTAAAAAGAATGCAGATTTTCAGAGAA...   
2666    SAS093         1  ATGGTAAAACGTACTTATCAACCAAATAAACGTAAACATAGTAAAG...   

                                       protein sequ

In [27]:
# high RNA expression value SAtmRNA01
reference1="GGGGACGTTCATGGATTCGACAGGGGTCCCCCGAGCTCATTAAGCGTGTCGGAGGGTTGGCTTCGTCATCAACACACACAGTTTATAATAACTGGCAAATCAAACAATAATTTCGCAGTAGCTGCCTAATCGCACTCTGCATCGCCTAACAGCATTTCCTATGTGCTGTTAACGCGATTCAACCTTAATAGGATATGCTAAACACTGCCGTTTGAAGTCTGTTTAGAAGAAACTTAATCAAGCTAGCATCATGTTGGTTGTTTATCACTTTTCATGATGCGAAACCTTTCGATAAACTACACACGTAGAAAGATGTGTATCAGGACCTCTGGACGCGGGTTCAAATCCCGCCGTCTCCAA"
# SAtRNA60

reference2="GGAGAATTAGCTCAGCTGGGAGAGCATCTGCCTTACAAGCAGAGGGTCGGCGGTTCGAACCCGTCATTCTCCACC"
cai_list=[]

for dna_sequence in N315_data['DNA']:
    try:
        cai=CAI(dna_sequence,reference=[reference1,reference2])
    except:
        cai=0.999
    cai_list.append(cai)
N315_data['CAI']=cai_list
N315_data = N315_data[N315_data['CAI'] != 0.999]

In [29]:
def calculate_gc_content(sequence):
    gc_count = sum(base.upper() in ['G', 'C'] for base in sequence)
    total_bases = len(sequence)
    gc_content = (gc_count / total_bases)
    return gc_content

N315_data['GC_Content'] = N315_data['DNA'].apply(calculate_gc_content)

print(N315_data)

     locus tag essential                                                DNA  \
0       SA0001         1  ATGTCGGAAAAAGAAATTTGGGAAAAAGTGCTTGAAATTGCTCAAG...   
1       SA0002         0  ATGATGGAATTCACTATTAAAAGAGATTATTTTATTACACAATTAA...   
2       SA0003         0  GTGATTATTTTGGTTCAAGAAGTTGTAGTAGAAGGAGACATTAATT...   
3       SA0004         1  ATGAAGTTAAATACACTCCAATTAGAAAATTATCGTAACTATGATG...   
4       SA0005         1  ATGGTGACTGCATTGTCAGATGTAAACAACACGGATAATTATGGTG...   
...        ...       ...                                                ...   
2661    SA2498         0  ATGAAAAAACCTTTTTCAAAATTATTTGGTTTGAAAAACAAAGATG...   
2662    SA2499         1  ATGACTGTAGAATGGTTAGCAGAACAATTAAAAGAACATAATATTG...   
2663    SA2500         1  GTGGTTCAAGAATATGATGTAATCGTTATAGGTGCGGGACATGCAG...   
2665    SA2502         1  TTGGAAAAAGCTTACCGAATTAAAAAGAATGCAGATTTTCAGAGAA...   
2666    SAS093         1  ATGGTAAAACGTACTTATCAACCAAATAAACGTAAACATAGTAAAG...   

                                       protein sequ

In [30]:
AminoAcidDict = {
    'A': 0, 'R': 0, 'N': 0, 'D': 0,
    'C': 0, 'Q': 0, 'E': 0, 'G': 0,
    'H': 0, 'I': 0, 'L': 0, 'K': 0,
    'M': 0, 'F': 0, 'P': 0, 'S': 0,
    'T': 0, 'W': 0, 'Y': 0, 'V': 0
}

def getAAFreq(protein_sequence):
    
    AminoAcids = AminoAcidDict.copy()

    for i in range(0, len(protein_sequence)):
        if protein_sequence[i] in AminoAcids:
            AminoAcids[protein_sequence[i]] += 1           

    FreqList = [AminoAcids[key]/len(protein_sequence) for key in AminoAcids]
 
    return FreqList

In [35]:
protein_sequence = []
for amino_acid in N315_data['protein sequence']:
    Freq=getAAFreq(amino_acid)
    protein_sequence.append(Freq)
amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
columns = {acid: [] for acid in amino_acids}

for sequence in protein_sequence:
    index = 0
    for acid in amino_acids:
        columns[acid].append(sequence[index])
        index+=1

result_df = pd.DataFrame(columns)

all_df_new = pd.concat([N315_data, result_df], axis=1)
print(all_df_new)
all_df_new.to_csv("../dataset/N315GC+CAI+Acimo.csv", index = False)

     locus tag essential                                                DNA  \
0       SA0001         1  ATGTCGGAAAAAGAAATTTGGGAAAAAGTGCTTGAAATTGCTCAAG...   
1       SA0002         0  ATGATGGAATTCACTATTAAAAGAGATTATTTTATTACACAATTAA...   
2       SA0003         0  GTGATTATTTTGGTTCAAGAAGTTGTAGTAGAAGGAGACATTAATT...   
3       SA0004         1  ATGAAGTTAAATACACTCCAATTAGAAAATTATCGTAACTATGATG...   
4       SA0005         1  ATGGTGACTGCATTGTCAGATGTAAACAACACGGATAATTATGGTG...   
...        ...       ...                                                ...   
2109       NaN       NaN                                                NaN   
2110       NaN       NaN                                                NaN   
2111       NaN       NaN                                                NaN   
2112       NaN       NaN                                                NaN   
2113       NaN       NaN                                                NaN   

                                       protein sequ

In [36]:
NCTC_data= pd.read_csv("../dataset/NCTC8325.csv")
# 替换值
NCTC_data['essential'] = NCTC_data['essential'].replace({'yes': 1, 'no': 0})

NCTC_data = NCTC_data[(NCTC_data['essential'] == 0) | (NCTC_data['essential'] == 1)]

print(NCTC_data)

          locus tag essential  \
1     SAOUHSC_00001         1   
3     SAOUHSC_00002         1   
4     SAOUHSC_00003         1   
5     SAOUHSC_00004         0   
6     SAOUHSC_00005         1   
...             ...       ...   
4108  SAOUHSC_03051         0   
4109  SAOUHSC_03052         1   
4110  SAOUHSC_03053         1   
4112  SAOUHSC_03054         1   
4114  SAOUHSC_03055         1   

                                                    DNA  \
1     ATGTCGGAAAAAGAAATTTGGGAAAAAGTGCTTGAAATTGCTCAAG...   
3     ATGATGGAATTCACTATTAAAAGAGATTATTTTATTACACAATTAA...   
4     GTGATTATTTTGGTTCAAGAAGTTGTAGTAGAAGGAGACATTAATT...   
5     ATGAAGTTAAATACACTCCAATTAGAAAATTATCGTAACTATGATG...   
6     ATGGTGACTGCATTGTCAGATGTAAACAACACGGATAATTATGGTG...   
...                                                 ...   
4108  ATGACTGTAGAATGGTTAGCAGAACAATTAAAAGAACATAATATTC...   
4109  GTGGTTCAAGAATATGATGTAATCGTTATAGGTGCGGGACATGCAG...   
4110  ATGGATTTAGATACAATTACGAGTATTTCAACACCAATGGGTGAAG...   
4112  ATGTTAT

In [37]:
# high RNA expression value SAOUHSC_01403
reference1="ATGAAACAAGGTACAGTTAAATGGTTTAACGCTGAAAAAGGATTCGGCTTTATCGAAGTTGAAGGAGAAAATGACGTATTCGTACATTTTTCAGCAATTAACCAAGATGGTTACAAATCATTAGAAGAAGGTCAAGCTGTTGAGTTTGAAGTAGTTGAAGGCGACCGCGGTCCACAAGCTGCAAACGTTGTTAAACTATAA"

# SAOUHSC_01191
reference2="ATGGGTAAACAATGTTTCGTAACAGGTCGTAAAGCTTCGACTGGTAACAGACGTTCACACGCTTTAAACTCTACTAAACGTAGATGGAACGCTAACCTTCAAAAAGTTAGAATCCTAGTTGACGGTAAACCTAAAAAAGTTTGGGTTTCTGCACGTGCTTTAAAATCTGGTAAAGTAACTAGAGTTTAA"
cai_list=[]

for dna_sequence in NCTC_data['DNA']:
    try:
        cai=CAI(dna_sequence,reference=[reference1,reference2])
    except:
        cai=0.999
    cai_list.append(cai)
NCTC_data['CAI']=cai_list
NCTC_data = NCTC_data[NCTC_data['CAI'] != 0.999]

In [39]:
NCTC_data['GC_Content'] = NCTC_data['DNA'].apply(calculate_gc_content)

In [41]:
print(NCTC_data)

          locus tag essential  \
1     SAOUHSC_00001         1   
3     SAOUHSC_00002         1   
4     SAOUHSC_00003         1   
5     SAOUHSC_00004         0   
6     SAOUHSC_00005         1   
...             ...       ...   
4108  SAOUHSC_03051         0   
4109  SAOUHSC_03052         1   
4110  SAOUHSC_03053         1   
4112  SAOUHSC_03054         1   
4114  SAOUHSC_03055         1   

                                                    DNA  \
1     ATGTCGGAAAAAGAAATTTGGGAAAAAGTGCTTGAAATTGCTCAAG...   
3     ATGATGGAATTCACTATTAAAAGAGATTATTTTATTACACAATTAA...   
4     GTGATTATTTTGGTTCAAGAAGTTGTAGTAGAAGGAGACATTAATT...   
5     ATGAAGTTAAATACACTCCAATTAGAAAATTATCGTAACTATGATG...   
6     ATGGTGACTGCATTGTCAGATGTAAACAACACGGATAATTATGGTG...   
...                                                 ...   
4108  ATGACTGTAGAATGGTTAGCAGAACAATTAAAAGAACATAATATTC...   
4109  GTGGTTCAAGAATATGATGTAATCGTTATAGGTGCGGGACATGCAG...   
4110  ATGGATTTAGATACAATTACGAGTATTTCAACACCAATGGGTGAAG...   
4112  ATGTTAT

In [46]:
protein_sequence = []
i=0
NCTC_data = NCTC_data.dropna(subset=['protein sequence'])
for amino_acid in NCTC_data['protein sequence']:
    Freq=getAAFreq(amino_acid)
    protein_sequence.append(Freq)

columns = {acid: [] for acid in amino_acids}

for sequence in protein_sequence:
    index = 0
    for acid in amino_acids:
        columns[acid].append(sequence[index])
        index+=1

result_df = pd.DataFrame(columns)

all_df_new = pd.concat([NCTC_data, result_df], axis=1)
print(all_df_new)
all_df_new.to_csv("../dataset/NCTCGC+CAI+Acimo.csv", index = False)

          locus tag essential  \
1     SAOUHSC_00001         1   
3     SAOUHSC_00002         1   
4     SAOUHSC_00003         1   
5     SAOUHSC_00004         0   
6     SAOUHSC_00005         1   
...             ...       ...   
2823            NaN       NaN   
2827            NaN       NaN   
2831            NaN       NaN   
2836            NaN       NaN   
2838            NaN       NaN   

                                                    DNA  \
1     ATGTCGGAAAAAGAAATTTGGGAAAAAGTGCTTGAAATTGCTCAAG...   
3     ATGATGGAATTCACTATTAAAAGAGATTATTTTATTACACAATTAA...   
4     GTGATTATTTTGGTTCAAGAAGTTGTAGTAGAAGGAGACATTAATT...   
5     ATGAAGTTAAATACACTCCAATTAGAAAATTATCGTAACTATGATG...   
6     ATGGTGACTGCATTGTCAGATGTAAACAACACGGATAATTATGGTG...   
...                                                 ...   
2823                                                NaN   
2827                                                NaN   
2831                                                NaN   
2836         